In [1]:
import awkward as ak
import uproot
import numpy as np
from yahist import Hist1D, Hist2D

In [2]:
import json
with open("./metadata/chunklist.json") as f:
    chunks = json.load(f) 
chunks.keys()

dict_keys(['EGamma_2018A', 'EGamma_2018B', 'EGamma_2018C', 'EGamma_2018D', 'DYJets', 'ttbar', 'ZG', 'WG', 'GJets_HT40To100', 'GJets_HT100To200', 'GJets_HT200To400', 'GJets_HT400To600', 'GJets_HT600ToInf', 'QCD_pT30To40', 'QCD_pT40ToInf', 'Diphoton', 'ZH', 'signal'])

In [3]:
chunks["signal"][0]

['/hadoop/cms/store/user/legianni/ProjectMetis/HHggtautau_Era2017____x3/test_nanoaodSkim_1.root',
 0,
 33831]

In [4]:
file = uproot.open(chunks["signal"][0][0])
tree = file['Events']
tree.keys()
#tree.show()
gen_keys = tree.keys(filter_name="GenPart_*")
genJet_keys = tree.keys(filter_name="GenJet_*")
gen_keys, genJet_keys

(['GenPart_eta',
  'GenPart_mass',
  'GenPart_phi',
  'GenPart_pt',
  'GenPart_genPartIdxMother',
  'GenPart_pdgId',
  'GenPart_status',
  'GenPart_statusFlags'],
 ['GenJet_eta',
  'GenJet_mass',
  'GenJet_phi',
  'GenJet_pt',
  'GenJet_partonFlavour',
  'GenJet_hadronFlavour'])

In [6]:
genPart = tree.arrays(gen_keys + genJet_keys + ["genWeight"], library="ak", how="zip") 
mask_genTau = abs(genPart.GenPart.pdgId) == 15
mask_genPho = abs(genPart.GenPart.pdgId) == 22 
mask_genH = genPart.GenPart.pdgId == 25
mIdx = genPart.GenPart.genPartIdxMother
mask_fromH = genPart.GenPart.pdgId[mIdx] == 25

print mask_fromH

In [44]:
genTau_fromH = genPart.GenPart[mask_genTau & mask_fromH]
genPho_fromH = genPart.GenPart[mask_genPho & mask_fromH]


HggIdx = genPho_fromH.genPartIdxMother[:,0]
# genH_gg = genPart.GenPart[:,HggIdx]
# # Hgg2d = np.s_[:,HggIdx]

# genH_gg = genPart.GenPart[Hgg2d]

HggIdx

<Array [16, 13, 15, 13, 12, ... 14, 11, 15, 13] type='33831 * int32'>

In [13]:
for el in genPho_fromH.genPartIdxMother:
    if len(el)!= 2:
        print (len(el))


In [125]:
print ("ho {} higgs e {} gammas su {} events".format(len(genH),len(genPho_fromH)*len(genPho_fromH.genPartIdxMother[1]),len(genPart)))

ho 33831 higgs e 67662 gammas su 33831 events


In [130]:
for genH in HggIdx:
    print (genH)

16
16


In [41]:
Hist1D(ak.num(genPho_fromH), bins=np.linspace(0,5,6))

bin,content
"(0,1)",0 ± 0
"(1,2)",0 ± 0
"(2,3)",33831 ± 183.932
"(3,4)",0 ± 0
"(4,5)",0 ± 0


In [29]:
Hist1D(ak.num(genTau_fromH), bins=np.linspace(0,5,6))

bin,content
"(0,1)",0 ± 0
"(1,2)",0 ± 0
"(2,3)",33831 ± 183.932
"(3,4)",0 ± 0
"(4,5)",0 ± 0


In [78]:
import numba as nb
import math

@nb.jit
def deltaphi_devfunc(phi1, phi2):
    dphi = phi1 - phi2
    out_dphi = float(0)
    
    if dphi > math.pi:
        dphi = dphi - 2 * math.pi
        out_dphi = dphi
    elif (dphi + math.pi) < 0:
        dphi = dphi + 2 * math.pi
        out_dphi = dphi
    else:
        out_dphi = dphi
    return out_dphi

In [79]:
%%time

@nb.jit
def get_deltaR(taus):
    nEvents = len(taus)
    dRs = np.empty(nEvents, np.float32)
    for i in range(nEvents):
        tau1 = taus[i][0]
        tau2 = taus[i][1]
        dR = np.sqrt((tau1.eta-tau2.eta)**2 + deltaphi_devfunc(tau1.phi, tau2.phi)**2)  
        dRs[i] = dR
    return dRs

dRs_ditau = get_deltaR(genTau_fromH)
dRs_dipho = get_deltaR(genPho_fromH)

CPU times: user 1.39 s, sys: 1.95 s, total: 3.33 s
Wall time: 2.54 s


In [126]:
print (len(genHgg.pt))
print (len(dRs_dipho))
for pt in genHgg.pt:
    print (len(pt))
    print (pt)

2
33831
48
[0, 0, 164, 164, 92.5, 314, 94.2, ... 3.53, 0.891, 0.334, 11.7, 12.3, 0.658, 0.206]
48
[0, 0, 164, 164, 92.5, 314, 94.2, ... 3.53, 0.891, 0.334, 11.7, 12.3, 0.658, 0.206]


In [108]:
len(dRs_dipho)
HptdRgg = ak.Array([dRs_dipho,genHgg.pt])

In [106]:
Hist2D((ak.flatten(genHgg.pt), dRs_dipho),bins=[np.linspace(0,500,51), np.linspace(0,4,41)])

Exception in thread Thread-20:
Traceback (most recent call last):
  File "/home/users/giacomo/miniconda3/envs/daskanalysisenv/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/home/users/giacomo/miniconda3/envs/daskanalysisenv/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/users/giacomo/miniconda3/envs/daskanalysisenv/lib/python3.7/site-packages/boost_histogram/_internal/hist.py", line 472, in fun
    local_hist.fill(*args, weight=weight, sample=sample)
ValueError: spans must have compatible lengths

Exception in thread Thread-16:
Traceback (most recent call last):
  File "/home/users/giacomo/miniconda3/envs/daskanalysisenv/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/home/users/giacomo/miniconda3/envs/daskanalysisenv/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/users/giacomo/miniconda3/envs/daskan

[[ 0.00 ± 0.00    0.00 ± 0.00    0.00 ± 0.00   ...    0.00 ± 0.00
   0.00 ± 0.00    0.00 ± 0.00]
 [ 0.00 ± 0.00    0.00 ± 0.00    0.00 ± 0.00   ...    0.00 ± 0.00
   0.00 ± 0.00    0.00 ± 0.00]
 [ 0.00 ± 0.00    0.00 ± 0.00    0.00 ± 0.00   ...    0.00 ± 0.00
   0.00 ± 0.00    0.00 ± 0.00]
 ...
 [ 0.00 ± 0.00    0.00 ± 0.00    0.00 ± 0.00   ...    0.00 ± 0.00
   0.00 ± 0.00    0.00 ± 0.00]
 [ 0.00 ± 0.00    0.00 ± 0.00    0.00 ± 0.00   ...    0.00 ± 0.00
   0.00 ± 0.00    0.00 ± 0.00]
 [ 0.00 ± 0.00    0.00 ± 0.00    0.00 ± 0.00   ...    0.00 ± 0.00
   0.00 ± 0.00    0.00 ± 0.00]]

In [37]:
Hist1D(dRs_ditau, bins=np.linspace(0,5,21))

[76.00 ± 8.72   774.00 ± 27.82   1596.00 ± 39.95   2478.00 ± 49.78
 3509.00 ± 59.24   4057.00 ± 63.69   4109.00 ± 64.10   3580.00 ± 59.83
 3190.00 ± 56.48   2716.00 ± 52.12   2346.00 ± 48.44   2199.00 ± 46.89
 1791.00 ± 42.32   678.00 ± 26.04   337.00 ± 18.36   175.00 ± 13.23
 94.00 ± 9.70   56.00 ± 7.48   31.00 ± 5.57   39.00 ± 6.24]